In [1]:
import numpy as np
import keras as k
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [20]:
"""
This function reads all of the files and lables from the json file then opens those files and converts the
images to np arrays to be fed into the nn.
NOTE: this is not a good thing to do if your dataset is too big, this may crash your computer
"""
# def imagegen():
f = open('models/InfraredSolarModules/module_metadata.json',) 
data = json.load(f) 
f.close() 
img = []
# cat = []
labels = []
for i in range(20000):
    img.append(np.array(k.preprocessing.image.load_img('models/InfraredSolarModules/' + data[str(i)]['image_filepath'], color_mode = "grayscale")))
    labels.append(data[str(i)]['anomaly_class'])
# return np.array(img), np.array(cat)

print("I done")

I done


In [48]:

labels[:10]

['Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module',
 'Offline-Module']

In [39]:
X_train, X_test, y_train, y_test = train_test_split(np.array(img), np.array(labels), test_size=0.2)
labelencoder = LabelEncoder() # Make a label encoder 
y_train = labelencoder.fit_transform(y_train) # encode labels to numbers
y_test = labelencoder.transform(y_test)

In [44]:
np.save('classes.npy', labelencoder.classes_)


In [40]:
print(np.unique(y_train))

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [32]:
np.unique(y_test)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [42]:
y_train = k.utils.to_categorical(y_train) # encode nubers to catagorical labels
y_test = k.utils.to_categorical(y_test)

In [37]:
np.unique(y_train)

array([0., 1.], dtype=float32)

In [43]:
labelencoder.inverse_transform([y_train[0]])

ValueError: bad input shape (1, 12)

In [8]:
X_train.shape

(16000, 40, 24)

In [9]:
y_train.shape

(16000, 12)

In [ ]:
# TODO: Upsample anomalies that don't happen often
counts = {}
for i in y_train:
    try:
        counts[str(i)] += 1
    except:
        counts[str(i)] = 0

In [ ]:
counts

In [ ]:
for i in range(len(y_train)):
    if counts[str(y_train[i])] < 1000:
        new_X_train = np.array([np.fliplr(X_train[i]), np.flipud(X_train[i]), np.flipud(np.fliplr(X_train[i]))])
        new_y_train = np.array([y_train[i], y_train[i], y_train[i]])
        X_train = np.concatenate((X_train, new_X_train), axis=0)
        y_train = np.concatenate((y_train, new_y_train), axis=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
model = k.Sequential()
model.add(k.layers.Conv2D(filters=15, kernel_size=(3,3), strides=(1, 1), padding="valid", input_shape=(40, 24, 1)))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Conv2D(filters=15, kernel_size=(3,3), strides=(1, 1), padding="valid"))
model.add(k.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="same"))
model.add(k.layers.Flatten())
model.add(k.layers.Dense(675))
model.add(k.layers.Activation('relu'))
model.add(k.layers.Dense(12))
model.add(k.layers.Activation('softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

In [ ]:
for i in range(20):
    print(i)
    model.fit(X_train, y_train, epochs=1, batch_size=20, verbose=1, shuffle=True)
    model.evaluate(X_test, y_test, verbose=1)

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Prediction loss, accuracy: {loss, acc}")

In [ ]:
model.save("model.h5")

In [ ]:
model = load_model('model.h5')